This notebook is some explorations of the symmetry group of the cube, driven by my need to understand it more for a project i'm working on.

In [ ]:
import itertools
import numpy
import pprint
import types

In [ ]:
pq = { "p" : 1, "q": -1 }
pqs = pq.items()
pqvs = pq.values()
# Place the cube with corners at +1/-1 in each coordinate. Make a handy namespace nc so i can refer to the corners as nc.ppp and so on.
nc = types.SimpleNamespace(**{ f"{x[0]}{y[0]}{z[0]}" : numpy.array([x[1], y[1], z[1]]) for x in pqs for y in pqs for z in pqs })
corners = [ corner for corner in nc.__dict__.values() ]
print(f"Set corners to {len(corners)} corners:")
pprint.pp(corners)
# Edges are a pair of corners that differ in exactly 1 coordinate. We name them like p_p for (ppp, pqp), _pq for (ppq, qpq), and so on, in namespace ne.
ne = types.SimpleNamespace(**{ e % (v[0],w[0]) : make(v,w) for e, make in {
    "_%s%s": lambda v,w:tuple(numpy.array([u, v[1], w[1]]) for u in pqvs),
    "%s_%s": lambda v,w:tuple(numpy.array([v[1], u, w[1]]) for u in pqvs),
    "%s%s_": lambda v,w:tuple(numpy.array([v[1], w[1], u]) for u in pqvs)
}.items() for v in pqs for w in pqs })
edges = [ edge for edge in ne.__dict__.values() ]
print(f"Set edges to {len(edges)} edges:")
pprint.pp(edges)
# Faces are four corners with 1 coordinate fixed. We name them like p__ for (ppp, ppq, pqq, pqp), and so on, in namespace nf.
# Note we use u*v as the second variable corner so that we walk the face in order.
nf = types.SimpleNamespace(**{ f % (w[0],) : make(w) for f, make in {
    "%s__": lambda w:tuple(numpy.array([w[1], u, u*v]) for u in pqvs for v in pqvs),
    "_%s_": lambda w:tuple(numpy.array([u, w[1], u*v]) for u in pqvs for v in pqvs),
    "__%s": lambda w:tuple(numpy.array([u, u*v, w[1]]) for u in pqvs for v in pqvs)
}.items() for w in pqs })
faces = [ face for face in nf.__dict__.values() ]
print(f"Set faces to {len(faces)} faces:")
pprint.pp(faces)
# Body diagonals are pairs of completely opposite corners. We name them with a manchester encoding for their coordinates, since opposite corners are opposite in every place
# I don't love rs for the encoding here, but i want something i can write as identifiers. Maybe s is same and r is rogue? I do want to avoid pq though.
sr = { "s" : 1, "r": -1 }
nd = types.SimpleNamespace(**{ d % (t[0],u[0]) : make(t,u) for d, make in {
    "%s%s": lambda t,u: tuple(numpy.array([v,v*t[1],v*t[1]*u[1]]) for v in pqvs)
}.items() for t in sr.items() for u in sr.items() })
diagonals = [ diagonal for diagonal in nd.__dict__.values() ]
print(f"Set diagonals to {len(diagonals)} diagonals:")
pprint.pp(diagonals)

In [ ]:
# Numpy equality is broadcast, so membership via in does work naively, so make some helpers
def corner_name(v):
    names = [ "p" if a == 1 else "q" if a == -1 else None for a in v]
    return None if None in names else "".join(names)
def is_a_corner(v):
    return corner_name(v) != None
def corner_id(v):
    name = corner_name(v)
    return None if name == None else list(nc.__dict__.keys()).index(name)

for c in corners:
    if not is_a_corner(c):
        print(f"FAIL: Corner c not is_a_corner")
        pprint.pp(c)

def edge_name(e):
    corner_names = [ corner_name(u) for u in e ]
    if None in corner_names:
        return None
    elif len(set(corner_names)) != 2:
        return None
    else:
        names = [t if t == u else "_" for t, u in zip(*corner_names)]
        if 1 != names.count("_"):
            return None
        else:
            return "".join(names)
def is_an_edge(e):
    return edge_name(e) != None
def edge_id(e):
    name = edge_name(e)
    return None if name == None else list(ne.__dict__.keys()).index(name)

for e in edges:
    if not is_an_edge(e):
        print(f"FAIL: Edge e not is_an_edge")
        pprint.pp(e)

def face_name(f):
    corner_names = [ corner_name(u) for u in f ]
    if None in corner_names:
        return None
    elif len(set(corner_names)) != 4:
        return None
    else:
        names = [ axis[0] if 1 == len(set(axis)) else "_" for axis in zip(*corner_names)]
        if len(names) - 1 != names.count("_"):
            return None
        else:
            # Check that the face isn't "distorted" - i.e. it's a sequence of edges.
            return "".join(names) if all([ is_an_edge([v, w]) for (v, w) in zip(f, f[1:] + f[:1]) ]) else None
def is_a_face(f):
    return face_name(f) != None
def face_id(f):
    name = face_name(f)
    return None if name == None else list(nf.__dict__.keys()).index(name)

for f in faces:
        if not is_a_face(f):
            print(f"FAIL: Face f not is_a_face")
            pprint.pp(f)

def diagonal_name(d):
    corner_names = [ corner_name(u) for u in d ]
    if None in corner_names:
        return None
    elif len(set(corner_names)) != 2:
        return None
    elif any(t == u for t, u in zip(*corner_names)):
        return None
    else:
        return "".join("s" if t == u else "r" for (t, u) in zip(corner_names[0][:-1], corner_names[0][1:]))
def is_a_diagonal(d):
    return diagonal_name(d) != None
def diagonal_id(d):
    name = diagonal_name(d)
    return None if name == None else list(nd.__dict__.keys()).index(name)

for d in diagonals:
        if not is_a_diagonal(d):
            print(f"FAIL: Diagonal d not is_a_diagonal")
            pprint.pp(d)

In [ ]:
print('nc = ' + pprint.pformat(nc))
print('ne = ' + pprint.pformat(ne))
print('nf = ' + pprint.pformat(nf))
print('nd = ' + pprint.pformat(nd))

The symmetry group of the cube permutes the faces. From this you can see it sends the centre of faces to the centre of faces. The centres of the faces are the vectors of unit length with precisely one coordinate nonzero, and the others zero - for example the centre of `p__` is $(1, 0, 0)$, the centre of `_q_` is $(0, -1, 0)$, and so on. Given a matrix $M$ which acts as a symmetry of the cube, we see then that
$$
\begin{pmatrix}
m_{11} & m_{12} & m_{13} \\
m_{21} & m_{22} & m_{23} \\
m_{31} & m_{32} & m_{33} \\
\end{pmatrix}
\times
\begin{pmatrix}
1 \\
0 \\
0 \\
\end{pmatrix}
=
\begin{pmatrix}
m_{11} \\
m_{21} \\
m_{31} \\
\end{pmatrix}
$$
so exactly one entry in the first column is nonzero, and it's either $1$ or $-1$. Similarly for the other columns. Since there are $3! = 6$ ways to arrange those entries, and $2^3 = 8$ ways to pick a sign for each nonzero entry, we get at most $6 \times 8 = 48$ symmetries. In fact, all of these are symmetries if you include reflections.

In [ ]:
symmetries = [numpy.array([p[0], p[1], p[2]]) for a in pqvs for b in pqvs for c in pqvs for p in itertools.permutations([(a,0,0),(0,b,0),(0,0,c)])]
print(f"Set symmetries to {len(symmetries)} symmetries")
# Do a quick check that everything we made preserves everything. Note that is_a_face checks the points are in a compatible order around the face
for S in symmetries:
    for v in corners:
        Sv = S@v
        if not is_a_corner(Sv):
            print("FAIL: S did not map corner v to another corner, follows are S, v, and S @ v")
            pprint.pp(S)
            pprint.pp(v)
            pprint.pp(Sv)
    for e in edges:
        Se = [S @ v for v in e]
        if not is_an_edge(Se):
            print("FAIL: S did not map edge e to another edge, follows are S, e, and S @ e[...]")
            pprint.pp(S)
            pprint.pp(e)
            pprint.pp(Se)
    for f in faces:
        Sf = [S @ v for v in f]
        if not is_a_face(Sf):
            print("FAIL: S did not map face f to another face, follows are S, f, and S @ f[...]")
            pprint.pp(S)
            pprint.pp(f)
            pprint.pp(Sf)

In [ ]:
def order(S):
    n = 1
    Sn = S
    I = numpy.identity(3)
    while not numpy.all(Sn == I):
        Sn = Sn @ S
        n = n + 1
    return n

In [ ]:
def face_permutation(S):
    return [(face_id(f), face_id([S @ v for v in f])) for f in faces]
def diagonal_permutation(S):
    return [(diagonal_id(d), diagonal_id([S @ v for v in d])) for d in diagonals]
def cycle_notation(permutation):
    p_as_map = [a[1] for a in sorted(permutation, key = lambda a: a[0])]
    untraced = list(range(len(permutation)))
    result = []
    while 0 != len(untraced):
        cycle = []
        next = untraced[0]
        while p_as_map[next] in untraced:
            cycle.append(next)
            next = p_as_map[next]
            untraced.remove(next)
        if len(cycle) > 1:
            result.append(cycle)
        # else omit trivial cycles
    return result

def cycle_sign(cycle):
    return "+" if sum(len(loop)-1 for loop in cycle) % 2 == 0 else "-"
    
def cycle_format(cycle):
    return "()" if 0 == len(cycle) else "".join(f"({"".join(f"{i}" for i in loop)})" for loop in cycle)

def gather_stats(S):
    face_perm = cycle_notation(face_permutation(S))
    return types.SimpleNamespace(**{
        "diag_perm":cycle_format(cycle_notation(diagonal_permutation(S))),
        "face_sign":cycle_sign(face_perm),
        "order":order(S),
        "det":int(numpy.linalg.det(S)),
        "face_perm":cycle_format(face_perm),
        "M":S
    })
        
stats = [gather_stats(S) for S in symmetries]
def columize(width, strings):
    count = 4
    total = len(strings)
    block = total // count
    if total != block * count:
        return None
    else:
        cols = [strings[i * block:(i+1)*block] for i in range(count)]
        return [[f"{str:^{width}}" for str in col] for col in cols]

def render_stat(stat):
  return "\n".join([l + r for l,r in zip(
    [f"D: {stat.diag_perm:14}", f"O: {stat.order:1} S: {stat.face_sign:1} d: {stat.det:2}  ", f"F: {stat.face_perm:14}" ],
    str(stat.M).split("\n")
  )])

stats = sorted(stats, key = lambda s: (-s.det,s.order,len(s.diag_perm), s.diag_perm))

I know for general reasons that the rotations have determinant 1 and the the reflections determinant -1. I was hoping to find a clean combinatorial way to tell them apart, but my first guess (the sign of the face permutation) is not correct.

In [ ]:
pprint.pp({ n : len([ stat for stat in stats if stat.order == n and stat.det == 1]) for n in sorted(set(stat.order for stat in stats))})

In [ ]:
for stat in stats:
    if stat.det == 1:
        print(render_stat(stat))
        print("")

In [ ]:
S = stats[18].M
print(render_stat(stats[18]))
for corner in corners:
    print(f"{corner_name(corner)} -> {corner_name(S @ corner)}")
for face in faces:
    print(f"{face_name(face)} -> {face_name([S @ v for v in face])}")
for diag in diagonals:
    print(f"{diagonal_name(diag)} -> {diagonal_name([S @ v for v in diag])}")

In [ ]:
import matplotlib.pyplot
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

axes = matplotlib.pyplot.figure().add_subplot(projection='3d')
axes.set_axis_off()

def barycentre(list):
    return sum(list)/len(list)

for face in faces:
    axes.add_collection3d(Poly3DCollection([face],edgecolors="black",linewidths=1,alpha=0.1,facecolors='grey'))
for corner in corners:
    axes.text(*corner,corner_name(corner),fontsize=18)
for diag in diagonals:
    axes.add_collection3d(Poly3DCollection([diag],edgecolors="black",linewidths=1))
    axes.text(*(diag[0]*0.8 + diag[1]*0.2),diagonal_name(diag),fontsize=14)

matplotlib.pyplot.show()